In [8]:
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt
from fiona.crs import from_epsg
import urllib, json, requests 
import geojson

In [9]:
from ipyleaflet import Map, GeoData, GeoJSON, basemaps, basemap_to_tiles, Icon, Circle, Marker, LayerGroup, WidgetControl
import ipywidgets as widgets
from ipywidgets import Button 
from IPython.display import display, clear_output

button = Button(description="Generate Text")
output = widgets.Output()

In [10]:
def find_location():
    lat = '40.7210907'
    lon = '-73.9877836'
    
    #Contruct request URL
    url = f'https://geo.fcc.gov/api/census/block/find?'

    # api call using variables instead of directly is way easier
    data_url = f'{url}lat={lat}&lon={lon}&format=json'
    #Get response from API
    response=requests.get(data_url)

    #Parse json in response
    data = response.json()
#     print(response.text)

    #Extracting the county and state being studied 
    state = data['results'][0]['state_fips']
    county_fips = data['results'][0]['county_fips']

    #Separating out the county fips from the state
    county = county_fips[2:]

#     print(state, county)
    
    return state

In [17]:
def download_acs():
    find_location()
    
    state = find_location()
    
    api_key = '9330dc4bf086a84f19fb412bb15f232507301de6'
    acs_url = f'https://api.census.gov/data/2018/acs/acs5/'

    pop_sex = 'B01003_001E,B01001_002E,B01001_026E'
    household_type = 'B11001_002E,B11001_007E'
    education = 'B15003_001E,B15003_017E,B15003_018E,B15003_019E,B15003_020E,B15003_021E,B15003_022E,B15003_023E,B15003_024E,B15003_025E'
    household_income = 'B19001_002E,B19001_003E,B19001_004E,B19001_005E,B19001_006E,B19001_007E,B19001_008E,B19001_009E,B19001_010E,B19001_011E,B19001_012E,B19001_013E,B19001_014E,B19001_015E,B19001_016E,B19001_017E'
    means_of_transport = 'B08301_002E,B08301_003E,B08301_003E,B08301_010E,B08301_016E,B08301_017E,B08301_018E,B08301_019E,B08301_020E,B08301_021E'
    travel_time = 'B08303_002E,B08303_003E,B08303_004E,B08303_005E,B08303_006E,B08303_007E,B08303_008E,B08303_009E,B08303_010E,B08303_011E,B08303_012E,B08303_013E'

    # api call using variables instead of directly is way easier
    pop_sex_url = f'{acs_url}?&get={pop_sex}&for=tract:*&in=state:{state}%20county:*&key={api_key}'
    household_type_url = f'{acs_url}?&get={household_type}&for=tract:*&in=state:{state}%20county:*&key={api_key}'
    education_url = f'{acs_url}?&get={education}&for=tract:*&in=state:{state}%20county:*&key={api_key}'
    household_income_url = f'{acs_url}?&get={household_income}&for=tract:*&in=state:{state}%20county:*&key={api_key}'
    means_of_transport_url = f'{acs_url}?&get={means_of_transport}&for=tract:*&in=state:{state}%20county:*&key={api_key}'
    travel_time_url = f'{acs_url}?&get={travel_time}&for=tract:*&in=state:{state}%20county:*&key={api_key}'

    response_acs1=requests.get(pop_sex_url)
    response_acs2=requests.get(household_type_url)
    response_acs3=requests.get(education_url)
    response_acs4=requests.get(household_income_url)
    response_acs5=requests.get(means_of_transport_url)
    response_acs6=requests.get(travel_time_url)
    # print(response_acs.text)

    #load the response into a JSON
    data_acs1=response_acs1.json()
    data_acs2=response_acs2.json()
    data_acs3=response_acs3.json()
    data_acs4=response_acs4.json()
    data_acs5=response_acs5.json()
    data_acs6=response_acs6.json()

    #ignoring the first element which is column names, replacing columns with the columns from the table
    acs1=pd.DataFrame(data_acs1[1:], columns=data_acs1[0])
    acs2=pd.DataFrame(data_acs2[1:], columns=data_acs2[0])
    acs3=pd.DataFrame(data_acs3[1:], columns=data_acs3[0])
    acs4=pd.DataFrame(data_acs4[1:], columns=data_acs4[0])
    acs5=pd.DataFrame(data_acs5[1:], columns=data_acs5[0])
    acs6=pd.DataFrame(data_acs6[1:], columns=data_acs6[0])
    
#     return acs1, acs2, acs3, acs4, acs5, acs6
 
    result = pd.DataFrame()

    result.append([acs1, acs2, acs3, acs4, acs5, acs6], ignore_index=False)
    return result

In [12]:
# def import_censustracts():
#     ##Adding in census geographies
#     state='STATE=36'

#     # census tracts link
#     endpoint = 'https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb/Tracts_Blocks/MapServer/4/query'
#     s = requests.session()
#     s.params = {
#         'where': 'STATE=36 AND COUNTY=',
#         'text': '',
#         'objectIds': '',
#         'time': '',
#         'geometry': '[[-2336502.536,-20037700.],[1.1615796196E7,2.0656504635E7]]',
#         'geometryType': 'esriGeometryEnvelope',
#         'inSR': 102100, # EPSG 3857
#         'spatialRel': 'esriSpatialRelIntersects',
#         'relationParam': '',
#     #specify fields
#         'outFields': 'GEOID,STATE,COUNTY,TRACT,STGEOMETRY, OBJECTID',
#         'returnGeometry': True,
#         'returnTrueCurves': False,
#         'maxAllowableOffset': '',
#         'geometryPrecision': '',
#         'outSR': '',
#         'returnIdsOnly': False,
#         'returnCountOnly': False,
#         'orderByFields': '',
#         'groupByFieldsForStatistics': '',
#         'outStatistics': '',
#         'returnZ': False,
#         'returnM': False,
#         'gdbVersion': '',
#         'returnDistinctValues': False,
#         'f': 'geojson',        
#     }
#     start = 0
#     done = False
#     features = []
#     crs = None
#     while not done:
#         r = s.get(endpoint, params={
#             'resultOffset': start,
#             'resultRecordCount': 32,
#         })
#         r.raise_for_status()
#         tracts = geojson.loads(r.text)
#         newfeats = tracts.__geo_interface__['features']
#         if newfeats:
#             features.extend(newfeats)
#             crs=tracts.__geo_interface__['crs']
#             start += len(newfeats)
#             print("Received", len(newfeats), "entries,", start, "total")
#         else:
#             done = True
#     tracts_df = gpd.GeoDataFrame.from_features(features, crs=crs)
    
#     return tracts_df

In [18]:
display(button, output)

def on_button_clicked(b):
    output.clear_output()
    
    test=find_location()
#     acs1, acs2, acs3, acs4, acs5, acs6 = download_acs()
#     tracts_df = import_censustracts()
    
#     return test
#     display(acs1, acs2, acs3, acs4, acs5, acs6) 
    result = download_acs()
    return result


    with output:
        print("Button clicked.")

button.on_click(on_button_clicked)

Button(description='Generate Text', style=ButtonStyle())

Output()

ValueError: Plan shapes are not aligned

TypeError: download_acs() got an unexpected keyword argument 'sort'

In [14]:
display(acs1)

NameError: name 'acs1' is not defined